In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=os.getenv("OPENROUTER_API_KEY",
))

response = client.embeddings.create(
  model="text-embedding-3-small",
  input="This is a test text for generating embeddings.")

response

: 